#### ollama Consensus Model

We start with a black-box approach to consensus generation. This has the downside that it works via a black-box so we have no way to understand how it is constructing a consensus. We also are not able to directly assess its performance. This idea is based off what was used in the Aware NLP model last semester.

In [1]:
import ollama
from ollama import pull, chat, generate
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, make_scorer, accuracy_score
from sklearn.metrics import recall_score, precision_score

In [8]:
ollama.pull('dolphin-mistral')
ollama.pull('mistral')

{'status': 'success'}

In [9]:
df= pd.read_csv('../data/highly_relevant_posts_descending_threshold_50.csv')

For sentiment analysis using this approach. I don't want to remove stopwords or punctuation, because that can drastically change the meaning of text (especially since 'not' is a stopword in the nltk corpus).

In [17]:
df['combined_text'] = df['combined_text'].str.replace('\n', ' ')

In [18]:
df.combined_text[1]

'What combination of meds finally worked for you? What hasn’t worked? right now i’m on: prozac 40mg wellbutrin xl 300mg adderall 15mg 2x a day xanax 0.5mg 1x day as needed  **meds not related to mental health** lyrica 75mg 2x per day vitamin d3 5000 omeprazole 20mg  i started seeing a psychiatrist and therapist in november. at that time i was put on prozac 20mg, topamax, and propranolol. i had a bad reaction to the topamax and switched to abilify which i had taken in the past. at some point in those few months we stopped the abilify, but i can’t remember why. we then increased the prozac to 40mg.   i had a hospitalization at the end of february where we did quite a few switches with my meds. the prozac stayed the same dose. they added wellbutrin xl 150mg, vistiril, and a low dose of seroquel (50mg i believe). during my week there we increased the dose of seroquel to 150mg and then by the end of the week i was at 300mg. i was then diagnosed with schizoaffective disorder. when i was disc

In [20]:
ollama_response = ollama.chat(model='dolphin-mistral', messages=[
  {
    'role': 'user',
    'content':  """ The following statement (delimited by ```) provided below is a post from the subreddit r/bpd. 
            The statement should be taken as is. It cannot be used to further a dialogue with the poster. 
            
            Please format the output as a dataframe with the columns "treatment", "sentiment", "current" and "dose". 
            The treatment column should give the names of the treatments and medications that the poster has used to treat borderline personality disorder.
            "sentiment" should give a list of the poster's experience with each of the treatments they discussed. For this, use the keys 'positive', 'negative', 'neutral' and 'unspecified'.
            "current" should either be blank or return a boolean value for whether the poster is currently recieving the treatment or not. If this cannot be determined, leave this field blank.
            Finally, if the post discusses a dosage associated to a medication, please list that in the "dose" column. Otherwise leave this blank.
            If the post discusses multiple doses, put all the doses separated by commas. Former doses should followed by '(former)'.
            <</SYS>>
            ```
            {df.combined_text[1]}
            ```,"""
  },
],
options = {
  #'temperature': 1.5, # very creative
  #'temperature': 0.1 # very conservative (good for coding and correct syntax)
})

# Printing the response from ollama.
print(ollama_response['message']['content'])

| treatment | sentiment | current | dose |
|---|---|---|---|
| Lithium | positive | | 600mg (current) |
| DBT | neutral | | |
| TMS | negative | | |
| Lexapro | positive | | 10mg (former), 20mg |
| Zoloft | unspecified | | |
| SSRIs | negative | | |
| SNRIs | neutral | | |
| Wellbutrin | neutral | | |


Refining this will require some prompt engineering. Also, this runs very slowly, so we probably want to minimize the amount of times we call this approach